In [1]:
!pip install duckdb
import duckdb
con = duckdb.connect(':memory:')
c = con.cursor()

     |████████████████████████████████| 1.3MB 2.7MB/s 
  Created wheel for duckdb: filename=duckdb-0.1.1-cp36-cp36m-linux_x86_64.whl size=1947911 sha256=8d9aaf3ef4866c8b0f31e9948e869652c178d644cd53ff9409e7d21bb118ea61
  Stored in directory: /root/.cache/pip/wheels/31/2e/81/8061e62cf80a0ea73a4657d5807c46a93105440af5921e828c
Successfully built duckdb


In [3]:
!wget -O Robust04.rar https://ucf779733eae3ffd3e1b0a0e596c.dl.dropboxusercontent.com/cd/0/get/AsNDmwlFgWQ-AJoSnA1RMAxwS18CT_PhhyTXrhdK__KP2YO4N-tqWpozg2Ky1ymktpSAPfario02XYTKxZqNNnfJ11M_E_lgEcfAAZVAco4Jtg5s0ioVFkBak-EyOl218Lw/file#
!unrar e -o+ Robust04.rar

--2019-11-12 08:05:57--  https://ucf779733eae3ffd3e1b0a0e596c.dl.dropboxusercontent.com/cd/0/get/AsNDmwlFgWQ-AJoSnA1RMAxwS18CT_PhhyTXrhdK__KP2YO4N-tqWpozg2Ky1ymktpSAPfario02XYTKxZqNNnfJ11M_E_lgEcfAAZVAco4Jtg5s0ioVFkBak-EyOl218Lw/file
Resolving ucf779733eae3ffd3e1b0a0e596c.dl.dropboxusercontent.com (ucf779733eae3ffd3e1b0a0e596c.dl.dropboxusercontent.com)... 162.125.82.6, 2620:100:6032:6::a27d:5206
Connecting to ucf779733eae3ffd3e1b0a0e596c.dl.dropboxusercontent.com (ucf779733eae3ffd3e1b0a0e596c.dl.dropboxusercontent.com)|162.125.82.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 436490487 (416M) [application/binary]
Saving to: ‘Robust04.rar’

Robust04.rar        100%[===================>] 416.27M  40.6MB/s    in 10s     

2019-11-12 08:06:08 (41.5 MB/s) - ‘Robust04.rar’ saved [436490487/436490487]


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from Robust04.rar

Extracting  docs.csv                                            

In [4]:
c.execute("CREATE TABLE dict(termid INTEGER "
                           ",term   VARCHAR "
                           ",df     INTEGER)")
c.execute("CREATE TABLE terms(termid INTEGER "
                            ",docid  INTEGER "
                            ",pos    INTEGER)")
c.execute("CREATE TABLE docs(name  VARCHAR "
                           ",docid INTEGER "
                           ",len   INTEGER "
                           ",temp  INTEGER)")   

In [5]:
c.execute("COPY dict "
          "FROM 'dict.csv' "
          "WITH DELIMITER '|'")
c.execute("COPY terms "
          "FROM 'terms.csv' "
          "WITH DELIMITER '|'")
c.execute("COPY docs "
          "FROM 'docs.csv' "
          "WITH DELIMITER '|'")

In [0]:
dict_head = c.execute("SELECT termid "
                            ",term "
                            ",df "
                      "FROM dict "
                      "ORDER BY RANDOM() "
                      "LIMIT 10").fetchdf()
terms_head = c.execute("SELECT termid "
                             ",docid "
                             ",pos "
                       "FROM terms "
                       "ORDER BY RANDOM() "
                       "LIMIT 10").fetchdf()
docs_head = c.execute("SELECT name "
                            ",docid "
                            ",len "
                       "FROM docs "
                       "ORDER BY RANDOM() "
                       "LIMIT 10").fetchdf()

In [0]:
dict_head

,termid,term,df
0,209151,"61,752",1
1,202955,"59,595",1
2,433863,hartigan,20
3,98366,"20,291",2
4,510886,masl,1
5,208101,"602,432",1
6,245168,823130,1
7,329276,caporali,1
8,233979,"77,708",1
9,395986,feldsher,3


In [0]:
terms_head

,termid,docid,pos
0,266033,260813,4
1,420786,325805,26
2,518310,15222,30
3,292865,163823,672
4,460695,240225,611
5,387670,307824,1886
6,682018,133524,1483
7,699046,131646,177
8,441148,136926,368
9,557714,40196,1719


In [0]:
docs_head

,name,docid,len
0,FT921-7152,31145,262
1,FT944-7115,393761,732
2,FBIS4-58815,114293,16
3,LA021190-0117,139010,74
4,LA042290-0149,268724,25
5,LA052289-0070,497846,63
6,FBIS4-37196,49172,179
7,FBIS3-10109,471160,318
8,FBIS4-14297,472003,100
9,LA010289-0015,467372,268


In [0]:
def make_query(*args: str):
    """Function to make query that works with SQL.
    
    Args:
        args: concatenation of strings to be made into a query
        
    Returns [2-tuple]:
        query     [str]: query in regex form \('.*'(, '.*')*\)
        len_query [str]: number of terms in the query as string
    """
    query = "('" + args[0] + "')"
    for arg in args[1:]:
        query = query[:-1] + ", '" + arg + "')"
        
    return query, str(len(args))

In [8]:
from datetime import datetime

query, len_query = make_query('wizard', 'hat')

cur_time = datetime.now()

qterms = c.execute("WITH qtermids "
                        "AS (SELECT dict.termid "
                                  ",dict.df "
                            "FROM dict "
                            "WHERE dict.term "
                                  "IN " + query +
                            ") "
                   "SELECT terms.termid "
                         ",terms.docid "
                         ",terms.pos "
                         ",qtermids.df "
                   "FROM terms "
                   "JOIN qtermids "
                   "ON terms.termid = qtermids.termid")

time_delta = datetime.now() - cur_time
print('Query time: {}'.format(time_delta))

qterms.fetchdf()

Query time: 0:00:00.821053


,termid,docid,pos,df
0,434301,44,589,1718
1,434301,152,497,1718
2,434301,281,92,1718
3,434301,339,1024,1718
4,434301,396,696,1718
...,...,...,...,...
2590,694497,323249,53,269
2591,694497,323249,186,269
2592,694497,324788,1341,269
2593,694497,326675,155,269


In [0]:
t = c.execute("WITH qtermids "
                   "AS (SELECT dict.termid "
                             ",dict.df "
                       "FROM dict "
                       "WHERE dict.term "
                             "IN " + query +
                       ") "
              ", qterms "
                   "AS (SELECT terms.termid "
                             ",terms.docid "
                             ",terms.pos "
                             ",qtermids.df "
                       "FROM terms "
                       "JOIN qtermids "
                       "ON terms.termid = qtermids.termid"
                       ") "
              "SELECT qterms.termid "
                    ",qterms.docid "
                    ",qterms.pos"
                    ",qterms.df "
              "FROM qterms")
t.fetchdf()

,termid,docid,pos,df
0,434301,44,589,1718
1,434301,152,497,1718
2,434301,281,92,1718
3,434301,339,1024,1718
4,434301,396,696,1718
...,...,...,...,...
2590,694497,323249,53,269
2591,694497,323249,186,269
2592,694497,324788,1341,269
2593,694497,326675,155,269


In [10]:
query, len_query = make_query('new', 'york')

t = c.execute("WITH qtermids "
                   "AS (SELECT dict.termid "
                             ",dict.df "
                       "FROM dict "
                       "WHERE dict.term "
                             "IN " + query +
                       ") "
              ", qterms "
                   "AS (SELECT terms.termid "
                             ",terms.docid "
                             ",terms.pos "
                             ",qtermids.df "
                       "FROM terms "
                       "JOIN qtermids "
                       "ON terms.termid = qtermids.termid"
                       ") "
              ", condocs "
                   "AS (SELECT qterms.docid "
                       "FROM qterms "
                       "GROUP BY qterms.docid "
                       "HAVING COUNT(DISTINCT qterms.termid) = " + len_query +
                       ") "
              "SELECT qterms.termid "
                    ",qterms.docid "
                    ",qterms.pos"
                    ",qterms.df "
              "FROM qterms "
              "JOIN condocs "
              "ON qterms.docid = condocs.docid")

t.fetchdf()

,termid,docid,pos,df
0,538933,8,276,163271
1,538933,8,368,163271
2,538933,8,481,163271
3,538933,8,571,163271
4,538933,8,706,163271
...,...,...,...,...
103742,708069,330730,183,20492
103743,708069,330730,590,20492
103744,708069,330737,204,20492
103745,708069,330176,39,20492


In [18]:
query, len_query = make_query('new', 'york')

t = c.execute("WITH qtermids "
                   "AS (SELECT dict.termid "
                             ",dict.df "
                       "FROM dict "
                       "WHERE dict.term "
                             "IN " + query +
                       ") "
              ", qterms "
                   "AS (SELECT terms.termid "
                             ",terms.docid "
                             ",terms.pos "
                             ",qtermids.df "
                       "FROM terms "
                       "JOIN qtermids "
                       "ON terms.termid = qtermids.termid"
                       ") "
              ", condocs "
                   "AS (SELECT qterms.docid "
                       "FROM qterms "
                       "GROUP BY qterms.docid "
                       "HAVING COUNT(DISTINCT qterms.termid) = " + len_query +
                       ") "
              "SELECT qterms1.termid AS termid1 "
                    ",qterms2.termid AS termid2 "
                    ",qterms1.docid "
                    ",qterms1.pos AS pos1 "
                    ",qterms2.pos AS pos2 "
                    ",1.0/(qterms1.pos - qterms2.pos) AS tpi "
                    ",qterms1.df AS df1 "
                    ",qterms2.df AS df2 "
                    ",CASE WHEN qterms1.df > qterms2.df THEN qterms1.df "
                          "ELSE qterms2.df "
                     "END AS maxdf "
              "FROM qterms AS qterms1 "
              "JOIN condocs "
              "ON qterms1.docid = condocs.docid "
              "JOIN qterms AS qterms2 "
              "ON qterms1.docid = qterms2.docid "
              "WHERE NOT qterms1.termid = qterms2.termid AND "
                    "qterms1.pos - qterms2.pos BETWEEN 1 AND 5")

t.fetchdf()

,termid1,termid2,docid,pos1,pos2,tpi,df1,df2,maxdf
0,538933,708069,614,2087,2086,1.000000,163271,20492,163271
1,538933,708069,614,2047,2044,0.333333,163271,20492,163271
2,538933,708069,568,1282,1277,0.200000,163271,20492,163271
3,538933,708069,1615,747,745,0.500000,163271,20492,163271
4,538933,708069,2962,1089,1084,0.200000,163271,20492,163271
...,...,...,...,...,...,...,...,...,...
35776,708069,538933,329810,41,40,1.000000,20492,163271,163271
35777,708069,538933,328801,51,50,1.000000,20492,163271,163271
35778,708069,538933,330112,109,108,1.000000,20492,163271,163271
35779,708069,538933,330176,39,38,1.000000,20492,163271,163271


In [20]:
query, len_query = make_query('new', 'york')

t = c.execute("WITH qtermids "
                   "AS (SELECT dict.termid "
                             ",dict.df "
                       "FROM dict "
                       "WHERE dict.term "
                             "IN " + query +
                       ") "
              ", qterms "
                   "AS (SELECT terms.termid "
                             ",terms.docid "
                             ",terms.pos "
                             ",qtermids.df "
                       "FROM terms "
                       "JOIN qtermids "
                       "ON terms.termid = qtermids.termid"
                       ") "
              ", condocs "
                   "AS (SELECT qterms.docid "
                       "FROM qterms "
                       "GROUP BY qterms.docid "
                       "HAVING COUNT(DISTINCT qterms.termid) = " + len_query +
                       ") "
              ", pairs "
                   "AS (SELECT qterms1.termid AS termid1 "
                             ",qterms2.termid AS termid2 "
                             ",qterms1.docid "
                             ",qterms1.pos AS pos1 "
                             ",qterms2.pos AS pos2 "
                             ",1.0/(qterms1.pos - qterms2.pos) AS tpi "
                             ",qterms1.df AS df1 "
                             ",qterms2.df AS df2 "
                             ",CASE WHEN qterms1.df > qterms2.df THEN qterms1.df "
                                   "ELSE qterms2.df "
                              "END AS maxdf "
                       "FROM qterms AS qterms1 "
                       "JOIN condocs "
                       "ON qterms1.docid = condocs.docid "
                       "JOIN qterms AS qterms2 "
                       "ON qterms1.docid = qterms2.docid "
                       "WHERE NOT qterms1.termid = qterms2.termid AND "
                             "qterms1.pos - qterms2.pos BETWEEN 1 AND 5"
                       ") "
              "SELECT pairs.termid1 "
                    ",pairs.termid2 "
                    ",pairs.docid "
                    ",SUM(pairs.tpi) AS tpisum "
              "FROM pairs "              
              "GROUP BY pairs.termid1 "
                      ",pairs.termid2 "
                      ",pairs.docid")

t.fetchdf()

,termid1,termid2,docid,tpisum
0,538933,708069,197293,0.5
1,538933,708069,216108,1.0
2,708069,538933,163973,1.0
3,708069,538933,239730,1.0
4,708069,538933,182864,1.0
...,...,...,...,...
20388,708069,538933,300446,1.0
20389,708069,538933,207230,1.0
20390,708069,538933,228742,1.0
20391,708069,538933,228587,1.0


In [22]:
query, len_query = make_query('new', 'york')

t = c.execute("WITH qtermids "
                   "AS (SELECT dict.termid "
                             ",dict.df "
                       "FROM dict "
                       "WHERE dict.term "
                             "IN " + query +
                       ") "
              ", qterms "
                   "AS (SELECT terms.termid "
                             ",terms.docid "
                             ",terms.pos "
                             ",qtermids.df "
                       "FROM terms "
                       "JOIN qtermids "
                       "ON terms.termid = qtermids.termid"
                       ") "
              ", condocs "
                   "AS (SELECT qterms.docid "
                       "FROM qterms "
                       "GROUP BY qterms.docid "
                       "HAVING COUNT(DISTINCT qterms.termid) = " + len_query +
                       ") "
              ", pairs "
                   "AS (SELECT qterms1.termid AS termid1 "
                             ",qterms2.termid AS termid2 "
                             ",qterms1.docid "
                             ",qterms1.pos AS pos1 "
                             ",qterms2.pos AS pos2 "
                             ",1.0/(qterms1.pos - qterms2.pos) AS tpi "
                             ",qterms1.df AS df1 "
                             ",qterms2.df AS df2 "
                             ",CASE WHEN qterms1.df > qterms2.df THEN qterms1.df "
                                   "ELSE qterms2.df "
                              "END AS maxdf "
                       "FROM qterms AS qterms1 "
                       "JOIN condocs "
                       "ON qterms1.docid = condocs.docid "
                       "JOIN qterms AS qterms2 "
                       "ON qterms1.docid = qterms2.docid "
                       "WHERE NOT qterms1.termid = qterms2.termid AND "
                             "qterms1.pos - qterms2.pos BETWEEN 1 AND 5"
                       ") "
              ",tpisums "
                   "AS (SELECT pairs.termid1 "
                             ",pairs.termid2 "
                             ",pairs.docid "
                             ",SUM(pairs.tpi) AS tpisum "
                       "FROM pairs "              
                       "GROUP BY pairs.termid1 "
                               ",pairs.termid2 "
                               ",pairs.docid"
                       ") "
              "SELECT pairs.termid1 "
                    ",pairs.termid2 "
                    ",pairs.docid "
                    ",tpisums.tpisum "
                    ",pairs.df1 "
                    ",pairs.df2 "
                    ",pairs.maxdf"
                    ",docs.len "
              "FROM pairs "
              "JOIN tpisums "
              "ON pairs.termid1 = tpisums.termid1 AND "
                 "pairs.termid2 = tpisums.termid2 AND "
                 "pairs.docid = tpisums.docid "
              "JOIN docs "
              "ON pairs.docid = docs.docid")

t.fetchdf()

,termid1,termid2,docid,tpisum,df1,df2,maxdf,len
0,538933,708069,2962,0.200000,163271,20492,163271,817
1,538933,708069,4043,0.333333,163271,20492,163271,844
2,538933,708069,5042,0.500000,163271,20492,163271,633
3,538933,708069,4887,0.500000,163271,20492,163271,777
4,538933,708069,4711,0.500000,163271,20492,163271,63
...,...,...,...,...,...,...,...,...
7060,708069,538933,328801,3.000000,20492,163271,163271,338
7061,708069,538933,328770,2.000000,20492,163271,163271,1277
7062,708069,538933,328801,3.000000,20492,163271,163271,338
7063,708069,538933,328801,3.000000,20492,163271,163271,338


In [25]:
query, len_query = make_query('new', 'york')

N = c.execute("SELECT COUNT(*) "
              "FROM docs")
N = str(N.fetchdf().iloc[0, 0])
avgdl = c.execute("SELECT AVG(docs.len) "
                  "FROM docs")
avgdl = str(avgdl.fetchdf().iloc[0, 0])

k1 = '1.2'
b = '0.75'

t = c.execute("WITH qtermids "
                   "AS (SELECT dict.termid "
                             ",dict.df "
                       "FROM dict "
                       "WHERE dict.term "
                             "IN " + query +
                       ") "
              ", qterms "
                   "AS (SELECT terms.termid "
                             ",terms.docid "
                             ",terms.pos "
                             ",qtermids.df "
                       "FROM terms "
                       "JOIN qtermids "
                       "ON terms.termid = qtermids.termid"
                       ") "
              ", condocs "
                   "AS (SELECT qterms.docid "
                       "FROM qterms "
                       "GROUP BY qterms.docid "
                       "HAVING COUNT(DISTINCT qterms.termid) = " + len_query +
                       ") "
              ", pairs "
                   "AS (SELECT qterms1.termid AS termid1 "
                             ",qterms2.termid AS termid2 "
                             ",qterms1.docid "
                             ",qterms1.pos AS pos1 "
                             ",qterms2.pos AS pos2 "
                             ",1.0/(qterms1.pos - qterms2.pos) AS tpi "
                             ",qterms1.df AS df1 "
                             ",qterms2.df AS df2 "
                             ",CASE WHEN qterms1.df > qterms2.df THEN qterms1.df "
                                   "ELSE qterms2.df "
                              "END AS maxdf "
                       "FROM qterms AS qterms1 "
                       "JOIN condocs "
                       "ON qterms1.docid = condocs.docid "
                       "JOIN qterms AS qterms2 "
                       "ON qterms1.docid = qterms2.docid "
                       "WHERE NOT qterms1.termid = qterms2.termid AND "
                             "qterms1.pos - qterms2.pos BETWEEN 1 AND 5"
                       ") "
              ",tpisums "
                   "AS (SELECT pairs.termid1 "
                             ",pairs.termid2 "
                             ",pairs.docid "
                             ",SUM(pairs.tpi) AS tpisum "
                       "FROM pairs "              
                       "GROUP BY pairs.termid1 "
                               ",pairs.termid2 "
                               ",pairs.docid"
                       ") "
              ", subscores "
                   "AS (SELECT pairs.termid1 "
                             ",pairs.termid2 "
                             ",pairs.docid "
                             ",tpisums.tpisum "
                             ",pairs.df1 "
                             ",pairs.df2 "
                             ",pairs.maxdf"
                             ",docs.len "
                             ",( LOG((" + N + "-maxdf+0.5)/(maxdf+0.5))*tpisum*(" + k1 + "+1)"
                                 "/"
                                "(tpisum+" + k1 + "*(1-" + b + "+" + b + "*len/" + avgdl + "))"
                               ") AS subscore "
                       "FROM pairs "
                       "JOIN tpisums "
                       "ON pairs.termid1 = tpisums.termid1 AND "
                          "pairs.termid2 = tpisums.termid2 AND "
                          "pairs.docid = tpisums.docid "
                       "JOIN docs "
                       "ON pairs.docid = docs.docid"
                       ") "
              "SELECT subscores.docid "
                    ",SUM(subscores.subscore) AS score "
              "FROM subscores "
              "GROUP BY subscores.docid")

t.fetchdf()

,docid,score
0,92886,0.259866
1,169982,0.267505
2,70503,0.808966
3,249123,0.252877
4,295096,2.424754
...,...,...
3803,270573,0.290361
3804,230826,0.320007
3805,66720,2.640376
3806,323301,1.683446


In [26]:
t = c.execute("WITH qtermids "
                   "AS (SELECT dict.termid "
                             ",dict.df "
                       "FROM dict "
                       "WHERE dict.term "
                             "IN " + query +
                       ") "
              ", qterms "
                   "AS (SELECT terms.termid "
                             ",terms.docid "
                             ",terms.pos "
                             ",qtermids.df "
                       "FROM terms "
                       "JOIN qtermids "
                       "ON terms.termid = qtermids.termid"
                       ") "
              ", condocs "
                   "AS (SELECT qterms.docid "
                       "FROM qterms "
                       "GROUP BY qterms.docid "
                       "HAVING COUNT(DISTINCT qterms.termid) = " + len_query +
                       ") "
              ", pairs "
                   "AS (SELECT qterms1.termid AS termid1 "
                             ",qterms2.termid AS termid2 "
                             ",qterms1.docid "
                             ",qterms1.pos AS pos1 "
                             ",qterms2.pos AS pos2 "
                             ",1.0/(qterms1.pos - qterms2.pos) AS tpi "
                             ",qterms1.df AS df1 "
                             ",qterms2.df AS df2 "
                             ",CASE WHEN qterms1.df > qterms2.df THEN qterms1.df "
                                   "ELSE qterms2.df "
                              "END AS maxdf "
                       "FROM qterms AS qterms1 "
                       "JOIN condocs "
                       "ON qterms1.docid = condocs.docid "
                       "JOIN qterms AS qterms2 "
                       "ON qterms1.docid = qterms2.docid "
                       "WHERE NOT qterms1.termid = qterms2.termid AND "
                             "qterms1.pos - qterms2.pos BETWEEN 1 AND 5"
                       ") "
              ",tpisums "
                   "AS (SELECT pairs.termid1 "
                             ",pairs.termid2 "
                             ",pairs.docid "
                             ",SUM(pairs.tpi) AS tpisum "
                       "FROM pairs "              
                       "GROUP BY pairs.termid1 "
                               ",pairs.termid2 "
                               ",pairs.docid"
                       ") "
              ", subscores "
                   "AS (SELECT pairs.termid1 "
                             ",pairs.termid2 "
                             ",pairs.docid "
                             ",tpisums.tpisum "
                             ",pairs.df1 "
                             ",pairs.df2 "
                             ",pairs.maxdf"
                             ",docs.len "
                             ",( LOG((" + N + "-maxdf+0.5)/(maxdf+0.5))*tpisum*(" + k1 + "+1)"
                                 "/"
                                "(tpisum+" + k1 + "*(1-" + b + "+" + b + "*len/" + avgdl + "))"
                               ") AS subscore "
                       "FROM pairs "
                       "JOIN tpisums "
                       "ON pairs.termid1 = tpisums.termid1 AND "
                          "pairs.termid2 = tpisums.termid2 AND "
                          "pairs.docid = tpisums.docid "
                       "JOIN docs "
                       "ON pairs.docid = docs.docid"
                       ") "
              ", scores "
                   "AS (SELECT subscores.docid "
                             ",SUM(subscores.subscore) AS score "
                       "FROM subscores "
                       "GROUP BY subscores.docid"
                       ") "
              "SELECT scores.docid "
                    ",scores.score "
              "FROM scores "
              "ORDER BY scores.score DESC "
              "LIMIT 100")

t.fetchdf()

,docid,score
0,50961,55.958108
1,19577,16.473072
2,181752,13.586938
3,139362,13.327779
4,238507,12.143474
...,...,...
95,65047,3.359299
96,82704,3.357105
97,182752,3.346258
98,223190,3.323418


In [32]:
cur_time = datetime.now()

t = c.execute("WITH qtermids "
                   "AS (SELECT dict.termid "
                             ",dict.df "
                       "FROM dict "
                       "WHERE dict.term "
                             "IN " + query +
                       ") "
              ", qterms "
                   "AS (SELECT terms.termid "
                             ",terms.docid "
                             ",terms.pos "
                             ",qtermids.df "
                       "FROM terms "
                       "JOIN qtermids "
                       "ON terms.termid = qtermids.termid"
                       ") "
              ", condocs "
                   "AS (SELECT qterms.docid "
                       "FROM qterms "
                       "GROUP BY qterms.docid "
                       "HAVING COUNT(DISTINCT qterms.termid) = " + len_query +
                       ") "
              ", pairs "
                   "AS (SELECT qterms1.termid AS termid1 "
                             ",qterms2.termid AS termid2 "
                             ",qterms1.docid "
                             ",1.0/(qterms1.pos - qterms2.pos) AS tpi "
                             ",( CASE WHEN qterms1.df > qterms2.df THEN qterms1.df "
                                     "ELSE qterms2.df "
                                "END "
                               ") AS maxdf "
                       "FROM qterms AS qterms1 "
                       "JOIN condocs "
                       "ON qterms1.docid = condocs.docid "
                       "JOIN qterms AS qterms2 "
                       "ON qterms1.docid = qterms2.docid "
                       "WHERE NOT qterms1.termid = qterms2.termid AND "
                             "qterms1.pos - qterms2.pos BETWEEN 1 AND 5"
                       ") "
              ",tpisums "
                   "AS (SELECT pairs.termid1 "
                             ",pairs.termid2 "
                             ",pairs.docid "
                             ",SUM(pairs.tpi) AS tpisum "
                       "FROM pairs "              
                       "GROUP BY pairs.termid1 "
                               ",pairs.termid2 "
                               ",pairs.docid"
                       ") "
              ", subscores "
                   "AS (SELECT pairs.docid "
                             ",( LOG((" + N + "-maxdf+0.5)/(maxdf+0.5))*tpisum*(" + k1 + "+1)"
                                 "/"
                                "(tpisum+" + k1 + "*(1-" + b + "+" + b + "*len/" + avgdl + "))"
                               ") AS subscore "
                       "FROM pairs "
                       "JOIN tpisums "
                       "ON pairs.termid1 = tpisums.termid1 AND "
                          "pairs.termid2 = tpisums.termid2 AND "
                          "pairs.docid = tpisums.docid "
                       "JOIN docs "
                       "ON pairs.docid = docs.docid"
                       ") "
              ", scores "
                   "AS (SELECT subscores.docid "
                             ",SUM(subscores.subscore) AS score "
                       "FROM subscores "
                       "GROUP BY subscores.docid"
                       ") "
              "SELECT scores.docid "
                    ",scores.score "
              "FROM scores "
              "ORDER BY scores.score DESC "
              "LIMIT 100")

time_delta = datetime.now() - cur_time
print("Query time: {}".format(time_delta))

t.fetchdf()

Query time: 0:00:05.302546


,docid,score
0,50961,55.958108
1,19577,16.473072
2,181752,13.586938
3,139362,13.327779
4,238507,12.143474
...,...,...
95,65047,3.359299
96,82704,3.357105
97,182752,3.346258
98,223190,3.323418
